In [10]:
import numpy as np
import onnx
import os
import glob
import onnxruntime.backend as backend

from onnx import numpy_helper

model = onnx.load('model.onnx')
test_data_dir = 'test_data_set_0'

# Load inputs
inputs = []
inputs_num = len(glob.glob(os.path.join(test_data_dir, 'input_*.pb')))
for i in range(inputs_num):
    input_file = os.path.join(test_data_dir, 'input_{}.pb'.format(i))
    tensor = onnx.TensorProto()
    with open(input_file, 'rb') as f:
        tensor.ParseFromString(f.read())
    inputs.append(numpy_helper.to_array(tensor))

# Load reference outputs
ref_outputs = []
ref_outputs_num = len(glob.glob(os.path.join(test_data_dir, 'output_*.pb')))
for i in range(ref_outputs_num):
    output_file = os.path.join(test_data_dir, 'output_{}.pb'.format(i))
    tensor = onnx.TensorProto()
    with open(output_file, 'rb') as f:
        tensor.ParseFromString(f.read())
    ref_outputs.append(numpy_helper.to_array(tensor))

# Run the model on the backend
outputs = list(backend.run(model, inputs))

# Compare the results with reference outputs.
for ref_o, o in zip(ref_outputs, outputs):
    np.testing.assert_almost_equal(ref_o, o, decimal = 7)

AssertionError: 
Arrays are not almost equal to 7 decimals

Mismatched elements: 6 / 10 (60%)
Max absolute difference: 0.00024414
Max relative difference: 3.6546427e-07
 x: array([[  975.6701  ,  -618.72394 ,  6574.5684  ,   668.02893 ,
         -917.27094 , -1671.6359  , -1952.7599  ,   -61.549873,
         -777.17664 , -1439.5316  ]], dtype=float32)
 y: array([[  975.6702  ,  -618.72394 ,  6574.5684  ,   668.0287  ,
         -917.27106 , -1671.6359  , -1952.76    ,   -61.549862,
         -777.17645 , -1439.5316  ]], dtype=float32)

In [4]:
# Compare the results with reference outputs.
for ref_o, o in zip(ref_outputs, outputs):
    np.testing.assert_allclose(ref_o, o)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 5 / 10 (50%)
Max absolute difference: 0.00024414
Max relative difference: 3.6546427e-07
 x: array([[  975.6701  ,  -618.72394 ,  6574.5684  ,   668.02893 ,
         -917.27094 , -1671.6359  , -1952.7599  ,   -61.549873,
         -777.17664 , -1439.5316  ]], dtype=float32)
 y: array([[  975.6702  ,  -618.72394 ,  6574.5684  ,   668.0287  ,
         -917.27106 , -1671.6359  , -1952.76    ,   -61.549862,
         -777.17645 , -1439.5316  ]], dtype=float32)

In [11]:
print(outputs)

[array([[  975.6702  ,  -618.72394 ,  6574.5684  ,   668.0287  ,
         -917.27106 , -1671.6359  , -1952.76    ,   -61.549862,
         -777.17645 , -1439.5316  ]], dtype=float32)]
